# Тестирование Web приложения

В этом коротком jupyter notebook предлагается провести легкое тестирование натсроенного веб приложения.

In [1]:
from datetime import datetime

In [2]:
import requests
import json

def post_request(uri, data_json):
    # Данная функция отправляет POST запрос и возвращает ответ сервера
    url = uri
    headers = {'Content-type': 'application/json',  # Определение типа данных
               'Accept': 'text/plain',
               'Content-Encoding': 'utf-8'}
    data = data_json  # Если по одному ключу находится несколько словарей, формируем список словарей
    answer = requests.post(url, data=json.dumps(data), headers=headers)
    print(answer)
    try:
        response = answer.json()
    except:
        response = answer.text
    
    print(response)

Укажите URL для функции `post_request`

In [3]:
base_url = 'http://localhost:8080'

Попробуем отправить на сервер список курьеров (они все валидные), должно вернуть JSON с их id

In [10]:
couriers = {"data": [
    {
        "courier_id": 1,
        "courier_type": "foot",
        "regions": [12, 22],
        "working_hours": ["11:35-14:05", "09:00-11:00"]
    },
    {
        "courier_id": 2,
        "courier_type": "bike",
        "regions": [22],
        "working_hours": ["09:00-18:00"]
    },
    {
        "courier_id": 3,
        "courier_type": "car",
        "regions": [12, 22, 23, 33],
        "working_hours": ["09:00-18:00"]
    },
    {
        "courier_id": 4,
        "courier_type": "bike",
        "regions": [12, 1],
        "working_hours": ["10:35-15:05", "17:00-19:00"]
    },
    {
        "courier_id": 5,
        "courier_type": "bike",
        "regions": [12],
        "working_hours": ["12:00-15:00"]
    }]}

In [11]:
post_request(base_url + '/couriers', couriers)

<Response [201]>
{'couriers': [{'id': 1}, {'id': 2}, {'id': 3}, {'id': 4}, {'id': 5}]}


Аналогично и для заказов

In [12]:
orders = {"data": [
    {
        "order_id": 1,
        "weight": 0.23,
        "region": 12,
        "delivery_hours": ["09:00-18:00"]
    },
    {
        "order_id": 2,
        "weight": 15,
        "region": 1,
        "delivery_hours": ["09:00-18:00"]
    },
    {
        "order_id": 3,
        "weight": 3.01,
        "region": 22,
        "delivery_hours": ["09:00-12:00", "16:00-21:30"]
    },
    {
        "order_id": 4,
        "weight": 5.7,
        "region": 1,
        "delivery_hours": ["17:00-21:00"]
    },
    {
        "order_id": 5,
        "weight": 0.01,
        "region": 22,
        "delivery_hours": ["18:00-21:30"]
    },
    {
        "order_id": 6,
        "weight": 12,
        "region": 12,
        "delivery_hours": ["12:00-18:00"]
    },
    {
        "order_id": 7,
        "weight": 4.71,
        "region": 12,
        "delivery_hours": ["09:00-12:00", "16:00-20:30"]
    }]}

In [13]:
post_request(base_url + '/orders', orders)

<Response [201]>
{'orders': [{'id': 1}, {'id': 2}, {'id': 3}, {'id': 4}, {'id': 5}, {'id': 6}, {'id': 7}]}


Курьер №3 имеет "courier_type": "car".

Присвоим ему заказы (должны присвоиться 'orders': [{'id': 1}, {'id': 3}, {'id': 7}, {'id': 6}])

In [14]:
assign = {"courier_id": 3}

In [15]:
post_request(base_url + '/orders/assign', assign)

<Response [200]>
{'assign_time': '2021-03-29 22:42:38.856332', 'orders': [{'id': 1}, {'id': 3}, {'id': 7}, {'id': 6}]}


Теперь попробуем изменить тип транспорта курьера №3 на пеший.

Часть заказов должна удалиться (id: 6)

In [17]:
update = {"courier_type": "foot"}

In [18]:
post_request(base_url + '/couriers/3', update)

<Response [200]>
{'courier_id': 3, 'courier_type': 'foot', 'regions': [12, 22, 23, 33], 'working_hours': ['09:00-18:00']}


И этот заказ снова доступен. Назначим его курьеру №5.

In [19]:
assign = {"courier_id": 5}
post_request(base_url + '/orders/assign', assign)

<Response [200]>
{'assign_time': '2021-03-29 22:43:02.152506', 'orders': [{'id': 6}]}


Также можем завершать заказы.

In [21]:
complete = {
"courier_id": 3,
"order_id": 1,
"complete_time": datetime.now().isoformat()
}

In [22]:
post_request(base_url + '/orders/complete', complete)

<Response [200]>
{'order_id': 1}


Теперь можно попробовать добавить невалидного курьера ("courier_type": "food").

Однако можно изменить файл (папка /data/openapi.yaml), поменяв или добавив новый тип курьера (если данный файл был проброшен при старте контейнера). Тогда этот курьер уже может быть признан валидным.

In [23]:
couriers = {"data": [
    {
        "courier_id": 9,
        "courier_type": "food",
        "regions": [12, 22],
        "working_hours": ["11:35-14:05", "09:00-11:00"]
    }]}

In [24]:
post_request(base_url + '/couriers', couriers)

<Response [400]>
{'validation_error': {'couriers': [{'id': 9}]}}
